In [1]:
import sys
import os
# Add the project root to the path so we can import from src
sys.path.append(os.path.abspath(".."))

import torch
from torch.utils.data import DataLoader
from src.datasets.shakespeare.shakespeare import ShakespeareDataset
from src.tokenizers.character_level.character_level import CharacterLevelTokenizer

## Configuration
Define the hyperparameters for the dataset and dataloader.

In [2]:
BATCH_SIZE = 32
MAX_LENGTH = 256
MIN_T = 1e-6
NUM_WORKERS = 0 

## Tokenizer and Dataset
Initialize the character-level tokenizer and the Shakespeare dataset.

In [3]:
tokenizer = CharacterLevelTokenizer()
dataset = ShakespeareDataset(tokenizer=tokenizer, max_length=MAX_LENGTH, min_t=MIN_T, train=True)

print(f"Dataset size: {len(dataset)}")
print(f"Vocab size: {tokenizer.vocab_size()}")

/media/john/Tertiary/Projects/ML/BayesianFlowNet/.venv/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for karpathy/tiny_shakespeare contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/karpathy/tiny_shakespeare
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset size: 1003854
Vocab size: 35


## Custom Collate Function
Define a collate function that packs the batch into a single sequence and returns document IDs.

In [4]:
def packed_collate_fn(batch):
    """
    Collate function to pack a batch of sequences into a single sequence.
    
    Args:
        batch: List of dictionaries, each containing 'x' (sequence) and 't' (timestep).
        
    Returns:
        Dictionary containing:
        - 'x': Packed sequence tensor of shape (total_seq_len,)
        - 'doc_ids': Document ID tensor of shape (total_seq_len,)
        - 't': Packed timestep tensor of shape (total_seq_len,)
    """
    xs = [item['x'] for item in batch]
    ts = [item['t'] for item in batch]
    
    # Pack x into a single sequence
    packed_x = torch.cat(xs, dim=0) # (total_seq_len,)
    
    # Create document ids
    doc_ids = []
    for i, x in enumerate(xs):
        doc_ids.append(torch.full_like(x, i))
    packed_doc_ids = torch.cat(doc_ids, dim=0) # (total_seq_len,)
    
    # Pack t (expand t for each token in the sequence)
    # t is (1,) per sample. We need to repeat it for len(x)
    packed_ts = []
    for i, (x, t) in enumerate(zip(xs, ts)):
        packed_ts.append(t.repeat(len(x)))
    packed_t = torch.cat(packed_ts, dim=0)

    return {
        "x": packed_x,
        "doc_ids": packed_doc_ids,
        "t": packed_t
    }

## DataLoader
Create the DataLoader using the custom collate function.

In [5]:
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=packed_collate_fn,
    num_workers=NUM_WORKERS
)

## Verification
Run a single batch to verify the shapes and content.

In [6]:
batch = next(iter(dataloader))
print("Batch keys:", batch.keys())
print("Packed x shape:", batch['x'].shape)
print("Doc ids shape:", batch['doc_ids'].shape)
print("Packed t shape:", batch['t'].shape)

print("\nSample check:")
print("First 10 tokens:", batch['x'][:10])
print("First 10 doc ids:", batch['doc_ids'][:10])
print("First 10 t values:", batch['t'][:10])

Batch keys: dict_keys(['x', 'doc_ids', 't'])
Packed x shape: torch.Size([8192])
Doc ids shape: torch.Size([8192])
Packed t shape: torch.Size([8192])

Sample check:
First 10 tokens: tensor([32, 22,  8, 19, 13,  4, 18, 18, 32, 19])
First 10 doc ids: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
First 10 t values: tensor([0.5833, 0.5833, 0.5833, 0.5833, 0.5833, 0.5833, 0.5833, 0.5833, 0.5833,
        0.5833])


## Model Components
Import necessary libraries and define the `PackDynamicSequenceChunker` and Flex Attention utilities.

In [7]:
from collections import namedtuple
import torch
from torch import cat, arange
from torch.nested import nested_tensor
from torch.nn import Module, Linear, Parameter
from torch.nn.functional import cosine_similarity, pad, softmax
from torch.nn.utils.rnn import pad_sequence
from einx import multiply
from einops import repeat, rearrange
from mamba_ssm import Mamba2
from torch.nn.attention.flex_attention import flex_attention, create_block_mask
from assoc_scan import AssocScan

# --- Helpers for Chunker ---

Outputs = namedtuple('Outputs', [
    'downsampled',
    'upsample_fn',
    'weighted_aux_ratio_loss'
])

Intermediates = namedtuple('Intermediates', [
    'mask',
    'probs',
    'chunk_lens',
    'boundary_mask',
    'residual',
    'gates',
    'upsampler_output_scale',
    'aux_ratio_loss',
    'input_mask',     # Added: needed to handle padding logic in upsampler
    'is_packed',      # Added: to track state
    'seq_lens',       # Added: original sequence lengths
    'new_seq_lens'    # Added: downsampled sequence lengths
])

def exists(v):
    return v is not None

def default(v, d):
    return v if exists(v) else d

def straight_through(t, value):
    return t + (value - t).detach()

def frac_gradient(t, frac = 1.):
    if frac == 1:
        return

    t_grad = t * frac
    return straight_through(t_grad, t)

# --- PackDynamicSequenceChunker ---

class PackDynamicSequenceChunker(Module):
    def __init__(
        self,
        dim,
        dim_queries_keys = None,
        boundary_threshold = 0.5,
        target_avg_token_length = 6.,       # N in eq(10)
        ratio_loss_weight = 3e-2,
        handle_residual_proj = False,       # turning this on will automatically handle a projection of the residual and its application in the inverse upsample function
        assoc_scan_use_accelerated = False,
        learning_rate_difference = 0.75,    # in the paper, they report that as one moves up a hierarchy, the learning rate needs to decrease. we'll default to 0.75 for the rough 2.0 -> 1.5 somewhere in the appendix from level 0 -> 1
        straight_through_frac_vecs = True,  # improvisation where F receives gradients through straight-through with sigmoid
    ):
        super().__init__()
        dim_queries_keys = default(dim_queries_keys, dim)

        # linear to queries and keys

        self.to_queries_keys = Linear(dim, dim_queries_keys * 2, bias = False)

        # start key token, so first token can be segmented / chunked out

        self.start_key_token = Parameter(torch.randn(dim_queries_keys) * 1e-2) # presumably, need a start key token for the first token, open an issue if i got it wrong

        # threshold to determine boundary

        assert 0. < boundary_threshold < 1.

        self.boundary_threshold = boundary_threshold

        # smoothing related

        self.smooth_assoc_scan = AssocScan(use_accelerated = assoc_scan_use_accelerated)

        # maybe residual proj

        self.handle_residual_proj = handle_residual_proj

        if handle_residual_proj:
            self.residual_proj = Linear(dim, dim)

        # learning rate modulation, appendix C
        # the multiplier on the learning rate as one goes from outer to inner of the h-net, and inverse of this value from inner to outer

        self.learning_rate_difference = learning_rate_difference

        # ratio aux loss related

        self.target_avg_token_length = target_avg_token_length

        self.straight_through_frac_vecs = straight_through_frac_vecs

        self.ratio_loss_weight = ratio_loss_weight

        self.register_buffer('zero', torch.tensor(0.), persistent = False)

    def upsample(
        self,
        downsampled,
        intermediates: Intermediates,
        apply_scale = True
    ):
        # handle packed sequence input for upsampling
        is_packed = downsampled.ndim == 2
        
        if is_packed:
            # If the original input was packed, the downsampled input here should be packed.
            # We need to unpack it to (B, N, D) to use the internal logic.
            assert exists(intermediates.new_seq_lens), "Cannot upsample packed sequence without new_seq_lens"
            downsampled_list = list(downsampled.split(intermediates.new_seq_lens.tolist()))
            downsampled = pad_sequence(downsampled_list, batch_first=True)

        batch, needs_grad, device = downsampled.shape[0], downsampled.requires_grad, downsampled.device

        mask = intermediates.mask
        gates = intermediates.gates
        residual = intermediates.residual

        # smoothing module for improved gradients eq(5)

        downsampled = self.smooth_assoc_scan(gates, downsampled)

        # upsample

        downsampled_without_padding = downsampled[mask]
        chunk_lens_without_padding = intermediates.chunk_lens[mask]

        seq = arange(downsampled_without_padding.shape[0], device = device)

        repeated_indices = torch.repeat_interleave(seq, chunk_lens_without_padding, dim = 0)
        upsampled = downsampled_without_padding[repeated_indices]

        upsampled = rearrange(upsampled, '(b n) d -> b n d', b = batch)

        scale = intermediates.upsampler_output_scale

        if needs_grad and apply_scale and exists(scale):
            upsampled = multiply('b n d, b n', upsampled, scale)

        if self.handle_residual_proj:
            # We need to use the original residual (input tokens)
            # If we are in packed mode, the residual stored in intermediates is likely padded.
            # We apply the projection on the padded residual and mask later if needed.
            upsampled = upsampled + self.residual_proj(residual)

        upsampled = frac_gradient(upsampled, self.learning_rate_difference)
        
        # If the original input was packed, we must return a packed sequence.
        # We use the original sequence lengths from intermediates to repack.
        if intermediates.is_packed:
            # Mask out padding from the padded upsampled result
            # intermediates.input_mask contains the valid locations of the original sequence
            if exists(intermediates.input_mask):
                upsampled = upsampled[intermediates.input_mask]
            else:
                # Fallback if mask missing (shouldn't happen in packed flow)
                upsampled_list = [upsampled[i, :l] for i, l in enumerate(intermediates.seq_lens)]
                upsampled = cat(upsampled_list, dim=0)

        return upsampled

    def forward(
        self,
        tokens, # float[b n d] OR float[total_n d] if packed
        seq_lens = None, # Required if tokens is packed
        return_intermediates = False,
        return_only_chunk_lens = False
    ):
        # ----------------------------------------------------------------------
        # Handle Packed Sequences
        # ----------------------------------------------------------------------
        is_packed = tokens.ndim == 2
        input_mask = None
        
        if is_packed:
            assert exists(seq_lens), 'seq_lens must be provided for packed sequences'
            # Unpack to (Batch, MaxLen, Dim) for processing
            tokens_list = list(tokens.split(seq_lens.tolist()))
            tokens_padded = pad_sequence(tokens_list, batch_first=True)
            
            # Create mask for padding
            # This is critical for cosine_sim and loss masking
            max_len = tokens_padded.shape[1]
            batch_size = len(seq_lens)
            range_tensor = arange(max_len, device=tokens.device).expand(batch_size, max_len)
            input_mask = range_tensor < seq_lens.unsqueeze(1)
            
            # Switch tokens pointer to padded version
            tokens = tokens_padded
        else:
            # Standard batched input
            # Create a default mask of all True if not provided
            batch, length = tokens.shape[:2]
            input_mask = torch.ones((batch, length), device=tokens.device, dtype=torch.bool)
            seq_lens = torch.full((batch,), length, device=tokens.device, dtype=torch.long)

        batch, length, device = *tokens.shape[:2], tokens.device

        residual = tokens

        queries, keys = self.to_queries_keys(tokens).chunk(2, dim = -1)

        start_keys = repeat(self.start_key_token, 'd -> b 1 d', b = batch)

        keys = cat((start_keys, keys), dim = 1)

        # each query looks at the previous key
        # Handle padding for cosine similarity: avoid NaN by adding epsilon to norms or masking
        
        keys_shifted = keys[:, :-1]
        
        # Safe cosine similarity
        eps = 1e-6
        queries_norm = queries.norm(dim=-1, keepdim=True)
        keys_norm = keys_shifted.norm(dim=-1, keepdim=True)
        numerator = (queries * keys_shifted).sum(dim=-1)
        denominator = (queries_norm * keys_norm).clamp(min=eps).squeeze(-1)
        cosine_sim = numerator / denominator

        # Mask out cosine sim on padding to avoid garbage boundaries
        cosine_sim = cosine_sim.masked_fill(~input_mask, 1.0) # 1.0 sim -> 0.0 prob -> no boundary

        probs = (1. - cosine_sim) * 0.5 # cosine sim is -1. to 1., this transforms it to 0. to 1.

        boundary_mask = probs > self.boundary_threshold # bool[b n]

        boundary_mask[:, 0] = True # first token must always be boundary
        
        # Ensure padding tokens are never boundaries
        boundary_mask = boundary_mask & input_mask

        # compute some lengths, per chunk and number of chunks per batch

        num_chunks = boundary_mask.long().sum(dim = -1)

        boundary_mask_with_end = pad(boundary_mask, (0, 1), value = True)
        sel_indices = repeat(arange(boundary_mask_with_end.shape[-1], device = device), 'n -> b n', b = batch)[boundary_mask_with_end]

        sel_indices = nested_tensor(sel_indices.split((num_chunks + 1).tolist()), layout = torch.jagged, device = device)

        sel_indices = sel_indices.to_padded_tensor(padding = -1)

        mask = (sel_indices != -1)[:, 1:]

        chunk_lens = sel_indices[:, 1:] - sel_indices[:, :-1]
        chunk_lens.masked_fill_(~mask, 0)

        # early return chunk lens if using a trained module as a tokenizer

        if return_only_chunk_lens:
            if is_packed:
                # Return packed chunk lenses
                return chunk_lens[mask]
            return chunk_lens

        # downsampling - they show in their experiments that picking out the boundary tokens works just fine

        boundary_tokens = tokens[boundary_mask] # pick out boundary tokens

        tokens_nt = nested_tensor(boundary_tokens.split(num_chunks.tolist()), layout = torch.jagged, device = device, requires_grad = True)

        downsampled_tokens = tokens_nt.to_padded_tensor(padding = 0.)

        # smoothing module for improved gradients eq(5)

        probs_nt = nested_tensor(probs[boundary_mask].split(num_chunks.tolist()), layout = torch.jagged, device = device, requires_grad = True)

        boundary_probs = probs_nt.to_padded_tensor(padding = 0.)

        gates = 1. - boundary_probs

        downsampled_tokens = multiply('b n d, b n', downsampled_tokens, boundary_probs)

        # for the upsampler

        confidence = torch.where(boundary_mask, probs, 1. - probs)

        # defaults if not training

        upsampler_output_scale = None
        aux_loss = self.zero
        weighted_aux_loss = self.zero

        needs_grad = tokens.requires_grad

        if needs_grad:
            # straight through for 1. multiplier on the expanded processed boundary tokens
            
            # For scale, we only care about valid tokens
            if is_packed:
                upsampler_output_scale = straight_through(confidence, 1.) # Padded shape
                # We don't mask here because multiply() in upsampler handles shape, 
                # but valid gradients only come from valid indices.
            else:
                upsampler_output_scale = straight_through(confidence, 1.)

            # auxiliary ratio loss in section 2.3.2, eq (10)

            N = self.target_avg_token_length

            F = boundary_mask.float()
            
            # Mask the probs for G calculation so padding doesn't affect mean
            probs_masked = probs * input_mask.float()
            G = probs_masked.sum(dim = -1) / input_mask.float().sum(dim = -1).clamp(min=1.)

            # allow for a soft F to straight through - https://arxiv.org/abs/2505.22074

            if self.straight_through_frac_vecs:
                F_soft = (probs - self.boundary_threshold).sigmoid()
                F = straight_through(F_soft, F)

            # Mask F for mean calculation
            F = (F * input_mask.float()).sum(dim = -1) / input_mask.float().sum(dim = -1).clamp(min=1.)

            aux_ratio_loss = N / (N - 1) * ((N - 1) * F * G + (1. - F) * (1. - G))

            aux_loss = aux_ratio_loss.mean()
            weighted_aux_loss = aux_loss * self.ratio_loss_weight

        # intermediates
        
        # Calculate new seq lengths for packed return
        new_seq_lens = num_chunks
        
        intermediates = Intermediates(
            mask, probs, chunk_lens, boundary_mask, residual, gates, 
            upsampler_output_scale, aux_loss,
            input_mask, is_packed, seq_lens, new_seq_lens
        )

        # return the upsample function

        def upsample(downsampled, apply_scale = True):
            return self.upsample(downsampled, intermediates, apply_scale = apply_scale)

        # adjust learning rate

        downsampled_tokens = frac_gradient(downsampled_tokens, self.learning_rate_difference ** -1)

        # handle packed output
        
        if is_packed:
            # We computed downsampled_tokens as (B, N_chunks, D) padded.
            # We need to pack it back to (Total_Chunks, D)
            # We can use new_seq_lens (num_chunks) to create the mask for valid data
            valid_chunks_mask = arange(downsampled_tokens.shape[1], device=device).expand(batch, -1) < new_seq_lens.unsqueeze(1)
            downsampled_tokens = downsampled_tokens[valid_chunks_mask]

        # returning

        outputs = Outputs(downsampled_tokens, upsample, weighted_aux_loss)

        if not return_intermediates:
            return outputs

        return outputs, intermediates

# --- Flex Attention Utils ---

def causal(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx

def generate_doc_mask_mod(mask_mod, document_id):
    # can feed in another mask modifier function such as `causal`

    # Get unique document IDs and their counts
    _, counts = torch.unique_consecutive(document_id, return_counts=True)
    # Create cumulative counts (offsets)
    offsets = torch.cat([torch.tensor([0], device=document_id.device), counts.cumsum(0)[:-1]])
    def doc_mask_wrapper(b, h, q_idx, kv_idx):
        same_doc = document_id[q_idx] == document_id[kv_idx]
        q_logical = q_idx - offsets[document_id[q_idx]]
        kv_logical = kv_idx - offsets[document_id[kv_idx]]
        inner_mask = mask_mod(b, h, q_logical, kv_logical)
        return same_doc & inner_mask
    return doc_mask_wrapper


## Model Definition
Define the `TransformerBlock` using Flex Attention and the `HybridModel` combining Mamba, H-Net Chunker, and Transformer.

In [8]:
class TransformerBlock(Module):
    def __init__(self, dim, heads, dim_head):
        super().__init__()
        self.norm1 = torch.nn.RMSNorm(dim)
        self.heads = heads
        self.dim_head = dim_head
        
        self.to_qkv = Linear(dim, heads * dim_head * 3, bias=False)
        self.to_out = Linear(heads * dim_head, dim, bias=False)
        
        self.norm2 = torch.nn.RMSNorm(dim)
        self.ff = torch.nn.Sequential(
            Linear(dim, dim * 4),
            torch.nn.GELU(),
            Linear(dim * 4, dim)
        )
        self.flex = torch.compile(flex_attention)

    def forward(self, x, block_mask):
        # x: (1, SeqLen, Dim) - treating packed as batch 1
        B, S, D = x.shape
        
        residual = x
        x = self.norm1(x)
        
        qkv = self.to_qkv(x) # (B, S, 3 * H * Dh)
        q, k, v = rearrange(qkv, 'b s (t h d) -> t b h s d', t=3, h=self.heads, d=self.dim_head)
        
        # Flex Attention
        out = self.flex(q, k, v, block_mask=block_mask) # (B, S, H, D)
        
        out = rearrange(out, 'b h s d -> b s (h d)')
        out = self.to_out(out)
        
        x = residual + out
        
        residual = x
        x = self.norm2(x)
        x = self.ff(x)
        x = residual + x
        
        return x

class HybridModel(Module):
    def __init__(self, dim, vocab_size, depth=4):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, dim)
        
        # Pre-chunk Mamba
        self.mamba_pre = Mamba2(
            d_model=dim,
            headdim=dim//16 if dim >= 16 else 4,
            d_state=16,
            d_conv=4,
            expand=2
        )
        
        # Chunker
        self.chunker = PackDynamicSequenceChunker(dim=dim)
        
        # Main Transformer (Flex Attention)
        self.transformer_blocks = torch.nn.ModuleList([
            TransformerBlock(dim, heads=4, dim_head=dim//4)
            for _ in range(depth)
        ])
        
        # Post-chunk Mamba
        self.mamba_post = Mamba2(
            d_model=dim,
            headdim=dim//16 if dim >= 16 else 4,
            d_state=16,
            d_conv=4,
            expand=2
        )
        
        self.head = Linear(dim, vocab_size)

    def forward(self, x, doc_ids):
        # x: (Total_L,)
        # doc_ids: (Total_L,)
        
        total_l = x.shape[0]

        # Embedding
        x = self.embedding(x) # (Total_L, Dim)
        
        # Mamba Pre
        # Treat as batch 1, but use seq_idx for packed logic
        x_unsqueezed = x.unsqueeze(0) # (1, Total_L, Dim)
        seq_idx = doc_ids.unsqueeze(0).int() # (1, Total_L)
        
        x = self.mamba_pre(x_unsqueezed, seq_idx=seq_idx) # (1, Total_L, Dim)
        x = x.squeeze(0) # (Total_L, Dim)
        
        # Chunker Downsample
        # Need seq_lens
        unique_doc_ids, counts = torch.unique_consecutive(doc_ids, return_counts=True)
        seq_lens = counts
        
        outputs, intermediates = self.chunker(x, seq_lens=seq_lens, return_intermediates=True)
        x_down = outputs.downsampled # (Total_Chunks, Dim)
        
        # Transformer
        # Reconstruct doc_ids for downsampled sequence
        with torch.no_grad():
            new_seq_lens = intermediates.new_seq_lens # (Batch_Size,)
            doc_ids_down = torch.repeat_interleave(unique_doc_ids, new_seq_lens)
        
        # Create mask for Flex Attention
        x_down_unsqueezed = x_down.unsqueeze(0) # (1, Total_Chunks, Dim)
        
        # Generate mask
        mask_mod = generate_doc_mask_mod(causal, doc_ids_down)
        block_mask = create_block_mask(mask_mod, B=None, H=None, Q_LEN=x_down_unsqueezed.shape[1], KV_LEN=x_down_unsqueezed.shape[1], device=x.device)
        
        for block in self.transformer_blocks:
            x_down_unsqueezed = block(x_down_unsqueezed, block_mask)
            
        x_down = x_down_unsqueezed.squeeze(0)
        
        # Chunker Upsample
        x_up = outputs.upsample_fn(x_down) # (Total_L, Dim)
        
        # Mamba Post
        x_up_unsqueezed = x_up.unsqueeze(0)
        x_final = self.mamba_post(x_up_unsqueezed, seq_idx=seq_idx)
        x_final = x_final.squeeze(0)
        
        # Head
        logits = self.head(x_final)
        return logits, outputs.weighted_aux_ratio_loss


## Model Verification
Instantiate the model and run a forward pass with the sample batch.

In [9]:
# Hyperparameters
DIM = 64
VOCAB_SIZE = tokenizer.vocab_size()
DEPTH = 2

# Instantiate Model
model = HybridModel(dim=DIM, vocab_size=VOCAB_SIZE, depth=DEPTH)
model = model.to('cuda')

# Get batch from previous step
x = batch['x'].to('cuda')
doc_ids = batch['doc_ids'].to('cuda')

print(f"Input shape: {x.shape}")
print(f"Doc IDs shape: {doc_ids.shape}")

Input shape: torch.Size([8192])
Doc IDs shape: torch.Size([8192])


In [10]:
# Forward Pass
logits, aux_loss = model(x, doc_ids)

print(f"Logits shape: {logits.shape}")
print(f"Aux Loss: {aux_loss.item()}")


/media/john/Tertiary/Projects/ML/BayesianFlowNet/.venv/lib/python3.12/site-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return _C._get_float32_matmul_precision()


Logits shape: torch.Size([8192, 35])
Aux Loss: 0.05101136863231659


In [11]:
aux_loss

tensor(0.0510, device='cuda:0', grad_fn=<MulBackward0>)

In [12]:
aux_loss.backward()

## Training Loop
Set up the optimizer and the training loop. The loop iterates through the dataloader, handles document boundaries for the loss, and updates the model.

In [13]:
from tqdm.auto import tqdm
# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(reduction='none') # We need to mask manually

# Training Loop
NUM_STEPS = 100
PRINT_EVERY = 10

model.train()
iter_dataloader = iter(dataloader)

print("Starting training...")

for step in tqdm(range(NUM_STEPS)):
    try:
        batch = next(iter_dataloader)
    except StopIteration:
        iter_dataloader = iter(dataloader)
        batch = next(iter_dataloader)
        
    x = batch['x'].to('cuda')
    doc_ids = batch['doc_ids'].to('cuda')
    
    # Prepare inputs and targets
    # We predict the next token
    input_ids = x[:-1]
    target_ids = x[1:]
    input_doc_ids = doc_ids[:-1]
    target_doc_ids = doc_ids[1:]
    
    # Forward pass
    logits, aux_loss = model(input_ids, input_doc_ids)
    
    # Calculate Loss
    ce_loss = criterion(logits, target_ids)
    
    # Mask loss at document boundaries
    # If input_doc_ids[i] != target_doc_ids[i], it means target is from a new doc
    # We shouldn't predict across documents
    valid_mask = (input_doc_ids == target_doc_ids).float()
    
    masked_ce_loss = (ce_loss * valid_mask).sum() / valid_mask.sum()
    
    total_loss = masked_ce_loss + aux_loss
    
    # Backward
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if step % PRINT_EVERY == 0:
        print(f"Step {step}: Loss {total_loss.item():.4f} (CE: {masked_ce_loss.item():.4f}, Aux: {aux_loss.item():.4f})")


Starting training...


  0%|          | 0/100 [00:00<?, ?it/s]

Step 0: Loss 3.6454 (CE: 3.5938, Aux: 0.0516)
Step 10: Loss 3.1206 (CE: 3.0875, Aux: 0.0331)
Step 20: Loss 3.0829 (CE: 3.0531, Aux: 0.0298)
Step 30: Loss 3.0995 (CE: 3.0691, Aux: 0.0304)
Step 40: Loss 3.0979 (CE: 3.0683, Aux: 0.0296)
Step 50: Loss 3.0527 (CE: 3.0211, Aux: 0.0316)
Step 60: Loss 3.0443 (CE: 3.0116, Aux: 0.0326)
Step 70: Loss 2.9868 (CE: 2.9515, Aux: 0.0353)
Step 80: Loss 2.9394 (CE: 2.9045, Aux: 0.0349)
Step 90: Loss 2.9026 (CE: 2.8656, Aux: 0.0370)
